# Nodal Basis

In [ ]:
from __future__ import division
#from sympy import *
import numpy as np
from numpy.linalg import *
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_basis(xVec, yVec, filename, plot_surplus=True, plot_interpolation=True, plot_points=True, color=None, figsize=None, hierarchical=False, tickz=False, boundaries=True):
    #first plot the training data
    if figsize is not None:
        fig, ax = plt.subplots(figsize=figsize)
    else:
        fig, ax = plt.subplots()

    if plot_points:
        plt.scatter(xVec,yVec, label="data", color="black")
    #the resulting function will just be the linear interpolation of the extended points
    if plot_interpolation:
        plt.plot(xVec,yVec,color="black",label="linear interpolation",linewidth=3,linestyle='--')
    #calculate the hat functions from before but now scale them with the height v_i
    if hierarchical:
        stride = 2
    else:
        stride = 1
    for i in range(1, len(xVec)-1, stride):
        if i == 1:
            x_1 = 0
        else:
            x_1 = xVec[i-1]
        if i == len(xVec)-2:
            x_3 = 1
        else:
            x_3 = xVec[i+1]
        xHat = [x_1,xVec[i],x_3]
        yHat = [0,yVec[i],0]
        if color is not None:
            plt.plot(xHat,yHat,color=color)
        else:
            plt.plot(xHat,yHat)
        if plot_surplus:
            plt.plot([xVec[i],xVec[i]],[0,yVec[i]], color="black")
    if hierarchical:
        if len(xVec) == 3:
            begin = 0
            end = 1
        else:
            boundaries = False
    else:
        begin = xVec[-2]
        end = xVec[1]
    if boundaries:
        if color is not None:
            plt.plot([0,end],[yVec[0],0],'--',color=color)
            plt.plot([begin,1],[0,yVec[-1]],'--',color=color)
        else:
            plt.plot([0,end],[yVec[0],0],'--')
            plt.plot([begin,1],[0,yVec[-1]],'--')
    
    plt.ylim([0, max(yVec)*1.2])
    if not tickz:
        ax.set_xticks([],[])
        ax.set_yticks([],[])
    plt.savefig(filename, bbox_inches='tight')
    plt.show()
xVec = np.linspace(0,1,9)

yVec = [x * (1-x) for x in xVec]
plot_basis(xVec, yVec, "nodal1D.pdf", tickz=True, boundaries=False)

In [ ]:
for l in range(1,4):
    xVec = np.linspace(0,1,2**l+1)
    yVec = np.ones(len(xVec))
    plot_basis(xVec,yVec, "nodal_basis_level_" + str(l) + ".pdf", plot_surplus=False, plot_interpolation=False, plot_points=False, color="black", figsize=(6,2))

# Hierarchical Basis

In [ ]:
for l in range(1,4):
    xVec = np.linspace(0,1,2**l+1)
    yVec = np.ones(len(xVec))
    plot_basis(xVec,yVec, "hierarchical_basis_level_" + str(l) + ".pdf", plot_surplus=False, plot_interpolation=False, plot_points=False, color="black", figsize=(6,2), hierarchical=True)

In [ ]:
from math import log2
#plt.plot(xVecExtended,yVecExtended,color="black",label="linear interpolation",linewidth=3,linestyle='--')
plt.scatter(xVec,yVec, label="data", color="black")
xVec = np.linspace(0,1,9)[1:-1]

yVec = [x * (1-x) for x in xVec]
numlevel = int(log2(len(xVec)+1))
yVecHier = list(yVec)
for l in range(numlevel):
    for j in range(2**l):
        offset = int(2**(numlevel - l))
        jLevel = offset//2 + j * offset - 1
        if jLevel - offset >= 0:
            leftPoint = xVec[jLevel - offset//2]
        else:
            leftPoint = 0
        if jLevel + offset < len(xVec):
            rightPoint = xVec[jLevel + offset//2]
        else:
            rightPoint = 1
        xHat = [leftPoint, xVec[jLevel], rightPoint]
        #print(jLevel)
        if l != 0:
            leftParent = jLevel - int(2**numlevel/2**(l+1))
            rightParent = jLevel + int(2**numlevel/2**(l+1))
            y = yVec[jLevel]
            yRight=yLeft=0
            if leftParent >= 0:
                y -= 0.5 * yVec[leftParent]
                yLeft = yVec[leftParent]
            if rightParent < len(yVec):
                y -= 0.5 * yVec[rightParent]
                yRight= yVec[rightParent]
            yVecHier[jLevel] = y
            yHat = [yLeft,yVec[jLevel],yRight]

        else:
            y = yVec[jLevel]
            yHat = [0,y,0]
        #print(xHat,yHat)
        plt.plot(xHat,yHat)
        plt.plot([xVec[jLevel],xVec[jLevel]], [yVec[jLevel], yVec[jLevel]-yVecHier[jLevel]], color="black")
        plt.ylim([0, 0.3])
plt.savefig("hierarchical1D.pdf", bbox_inches='tight')

In [ ]:
from math import log2
#plt.plot(xVecExtended,yVecExtended,color="black",label="linear interpolation",linewidth=3,linestyle='--')
#plt.scatter(xVec,yVec, label="data", color="black")

numlevel = int(log2(len(xVec)+1))
yVecHier = list(yVec)
for l in range(numlevel):
    for j in range(2**l):
        offset = int(2**(numlevel - l))
        jLevel = offset//2 + j * offset - 1
        if jLevel - offset >= 0:
            leftPoint = xVec[jLevel - offset//2]
        else:
            leftPoint = 0
        if jLevel + offset < len(xVec):
            rightPoint = xVec[jLevel + offset//2]
        else:
            rightPoint = 1
        xHat = [leftPoint, xVec[jLevel], rightPoint]
        #print(jLevel)
        if l != 0:
            leftParent = jLevel - int(2**numlevel/2**(l+1))
            rightParent = jLevel + int(2**numlevel/2**(l+1))
            y = yVec[jLevel]
            yRight=yLeft=0
            if leftParent >= 0:
                y -= 0.5 * yVec[leftParent]
                yLeft = yVec[leftParent]
            if rightParent < len(yVec):
                y -= 0.5 * yVec[rightParent]
                yRight= yVec[rightParent]
            yVecHier[jLevel] = y
            yHat = [0,yVecHier[jLevel],0]

        else:
            y = yVec[jLevel]
            yHat = [0,y,0]
        #print(xHat,yHat)
        plt.plot(xHat,yHat)
        plt.plot([xVec[jLevel],xVec[jLevel]], [0, yVecHier[jLevel]], color="black")
        plt.ylim([0, 0.3])
plt.savefig("hierarchical1Dflat.pdf", bbox_inches='tight')

In [ ]:
%matplotlib inline
import sparseSpACE
from sparseSpACE.Function import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.Grid import *
import numpy as np
dim = 2
a = np.zeros(dim)
b = np.ones(dim)
grid = TrapezoidalGrid(a=a, b=b, boundary=True)
f = FunctionExpVar()
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = f.getAnalyticSolutionIntegral(a,b))
combiObject = StandardCombi(a, b, operation=operation)
minimum_level = 1
maximum_level = 3
combiObject.set_combi_parameters(minimum_level, maximum_level)
combiObject.print_subspaces(sparse_grid_spaces=False, ticks=False, fade_full_grid=False, filename="hierarchical_subspaces.pdf")

# Sparse Grid

In [ ]:
%matplotlib inline
import sparseSpACE
from sparseSpACE.Function import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.Grid import *
import numpy as np
dim = 2
a = np.zeros(dim)
b = np.ones(dim)
grid = TrapezoidalGrid(a=a, b=b, boundary=True)
f = FunctionExpVar()
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = f.getAnalyticSolutionIntegral(a,b))
combiObject = StandardCombi(a, b, operation=operation)
minimum_level = 1
maximum_level = 3
combiObject.set_combi_parameters(minimum_level, maximum_level)
combiObject.print_subspaces(sparse_grid_spaces=False, ticks=False, fade_full_grid=True, filename="sparsegrid_subspaces.pdf")

In [ ]:
combiObject.print_resulting_sparsegrid(ticks=False, show_border=True, filename="sparsegrid.pdf")

In [ ]:
%matplotlib inline
import sparseSpACE
from sparseSpACE.Function import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.Grid import *
import numpy as np
dim = 2
a = np.zeros(dim)
b = np.ones(dim)
grid = ClenshawCurtisGrid(a=a, b=b, boundary=True)
f = FunctionExpVar()
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = f.getAnalyticSolutionIntegral(a,b))
combiObject = StandardCombi(a, b, operation=operation)
minimum_level = 1
maximum_level = 3
combiObject.set_combi_parameters(minimum_level, maximum_level)
combiObject.print_resulting_sparsegrid(ticks=False, show_border=True, filename="sparsegrid_chebyshev.pdf")

In [ ]:
%matplotlib inline
import sparseSpACE
from sparseSpACE.Function import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.Grid import *
import numpy as np
dim = 2
a = np.zeros(dim)
b = np.ones(dim)
grid = TrapezoidalGrid(a=a, b=b, boundary=True)
f = FunctionExpVar()
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = f.getAnalyticSolutionIntegral(a,b))
combiObject = StandardCombi(a, b, operation=operation)
minimum_level = 1
maximum_level = 3
combiObject.set_combi_parameters(minimum_level, maximum_level)
additional_points = [(0.25,0.125), (0.25,0.375), (0.125,0.25), (0.375,0.25), (0.5,0.9375), (0.5,0.8125), (0.25,0.875), (0.75,0.875)]
combiObject.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_adaptive.pdf", additional_points=additional_points)

# Combination Technique

In [ ]:
%matplotlib inline
import sparseSpACE
from sparseSpACE.Function import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.Grid import *
import numpy as np
dim = 2
a = np.zeros(dim)
b = np.ones(dim)
grid = TrapezoidalGrid(a=a, b=b, boundary=True)
f = FunctionExpVar()
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = f.getAnalyticSolutionIntegral(a,b))
combiObject = StandardCombi(a, b, operation=operation)
minimum_level = 1
maximum_level = 3
combiObject.set_combi_parameters(minimum_level, maximum_level)
combiObject.print_resulting_combi_scheme(ticks=False, filename="combi.pdf")

# Dimension Adaptive Combination Technique

In [ ]:
%matplotlib inline
import sparseSpACE
from sparseSpACE.Function import *
from sparseSpACE.DimAdaptiveCombi import *
from sparseSpACE.Grid import *
import numpy as np
dim = 2
coeffs = np.ones(dim)
a = np.zeros(dim)
b = np.ones(dim)
grid = TrapezoidalGrid(a=a, b=b)
f = GenzCornerPeak(coeffs=coeffs)
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = f.getAnalyticSolutionIntegral(a,b))
combiObject = DimAdaptiveCombi(a, b, operation=operation)
minimum_level = 1
maximum_level = 2
combiObject.perform_combi(minimum_level, maximum_level, 2*10**-3)
combiObject.print_resulting_combi_scheme(ticks=False, filename="combi_dimadaptive.pdf", fontsize=60)
combiObject.print_resulting_sparsegrid(ticks=False, show_border=True, filename="sparsegrid_dimadaptive.pdf")

# SpatiallyAdaptiveCombi

# Split-Extend

In [ ]:
%matplotlib inline
import sparseSpACE
import numpy as np
from sparseSpACE.spatiallyAdaptiveExtendSplit import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *


# dimension of the problem
dim = 2

# define integration domain boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function to be integrated
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**0 * (d+1) for d in range(dim)])
f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# reference integral solution for calculating errors
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# define error estimator for refinement
errorOperator2=ErrorCalculatorExtendSplit()

# define equidistant grid
grid=TrapezoidalGrid(a=a, b=b)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = reference_solution)

# define Extend-Split refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceExtend = SpatiallyAdaptiveExtendScheme(a, b, 3, version=0, operation=operation)

# performing the spatially adaptive refinement with the Extend-Split method
adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(1, 2, errorOperator2, 10**-2, max_evaluations=0, do_plot=False)


print("Number of points used in refinement:", adaptiveCombiInstanceExtend.get_total_num_points())
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("combi_initial_split.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("combi_initial_split.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)


# Split

In [ ]:
%matplotlib inline
import numpy as np
import sparseSpACE
from sparseSpACE.spatiallyAdaptiveExtendSplit import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**0 * (d+1) for d in range(dim)])
f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# reference integral solution for calculating errors
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# define error estimator for refinement
errorOperator=ErrorCalculatorExtendSplit()

# define equidistant grid
grid=TrapezoidalGrid(a=a, b=b)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = reference_solution)

# define Extend-Split refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceExtend = SpatiallyAdaptiveExtendScheme(a, b, 3, version=0, operation=operation)

# perform spatially adaptive integration
adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=2*10**-0, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_initial.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_initial.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_split_initial.pdf")

adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=2*10**-1, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_1.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_1.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_split_1.pdf")

adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=1.6*10**-1, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_2.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_2.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_split_2.pdf")


In [ ]:
%matplotlib inline
import numpy as np
import sparseSpACE
from sparseSpACE.spatiallyAdaptiveExtendSplit import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**0 * (d+1) for d in range(dim)])
f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# reference integral solution for calculating errors
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# define error estimator for refinement
errorOperator=ErrorCalculatorExtendSplit()

# define equidistant grid
grid=TrapezoidalGrid(a=a, b=b)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = reference_solution)

# define Extend-Split refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceExtend = SpatiallyAdaptiveExtendScheme(a, b, 100, version=0, operation=operation, split_single_dim=True)

# perform spatially adaptive integration
#adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=2*10**-0, do_plot=False)

#adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_initial.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
#adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_initial.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
#adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_split_initial.pdf")

adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=2*10**-1, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_single_1.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_single_1.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_split_single_1.pdf")

adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=1.6*10**-1, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_single_2.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("split_single_2.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_split_single_2.pdf")


# Extend

In [ ]:
%matplotlib inline
import numpy as np
import sparseSpACE
from sparseSpACE.spatiallyAdaptiveExtendSplit import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**0 * (d+1) for d in range(dim)])
f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# reference integral solution for calculating errors
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# define error estimator for refinement
errorOperator=ErrorCalculatorExtendSplit()

# define equidistant grid
grid=TrapezoidalGrid(a=a, b=b)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = reference_solution)

# define Extend-Split refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceExtend = SpatiallyAdaptiveExtendScheme(a, b, number_of_refinements_before_extend=0,operation=operation,version=0)

# perform spatially adaptive integration
adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=2*10**-0, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_initial.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_initial.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_extend_initial.pdf")

adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=2*10**-1, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_1.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_1.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_extend_1.pdf")

adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=1.6*10**-1, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_2.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_2.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_extend_2.pdf")


In [ ]:
%matplotlib inline
import numpy as np
import sparseSpACE
from sparseSpACE.spatiallyAdaptiveExtendSplit import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**0 * (d+1) for d in range(dim)])
f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# reference integral solution for calculating errors
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# define error estimator for refinement
errorOperator=ErrorCalculatorExtendSplit()

# define equidistant grid
grid=TrapezoidalGrid(a=a, b=b)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = reference_solution)

# define Extend-Split refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceExtend = SpatiallyAdaptiveExtendScheme(a, b, number_of_refinements_before_extend=0,operation=operation,version=3)

# perform spatially adaptive integration
adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=2*10**-0, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_noNull_initial.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_noNull_initial.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_noNull_extend_initial.pdf")

adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=2*10**-1, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_noNull_1.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_noNull_1.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_noNull_extend_1.pdf")

adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=1.6*10**-1, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_noNull_2.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("extend_noNull_2.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_sparsegrid(ticks=True, show_border=True, filename="sparsegrid_noNull_extend_2.pdf")


# Error estimate 

In [ ]:
%matplotlib inline
import numpy as np
import sparseSpACE
from sparseSpACE.spatiallyAdaptiveExtendSplit import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 0.4*10**1  for d in range(dim)])
f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# reference integral solution for calculating errors
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# define error estimator for refinement
errorOperator=ErrorCalculatorExtendSplit()

# define equidistant grid
grid=TrapezoidalGrid(a=a, b=b)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution = reference_solution)

# define Extend-Split refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceExtend = SpatiallyAdaptiveExtendScheme(a, b, number_of_refinements_before_extend=2,operation=operation,version=0)

# perform spatially adaptive integration
adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=0, max_evaluations=0, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("error_estimate_initial.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("error_estimate_initial.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)

adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=0, max_evaluations=22, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("error_estimate_split.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("error_estimate_split.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)

adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,tol=0, max_evaluations=50, do_plot=False)

adaptiveCombiInstanceExtend.print_resulting_combi_scheme("error_estimate_both.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("error_estimate_both.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)

adaptiveCombiInstanceExtend = SpatiallyAdaptiveExtendScheme(a, b, number_of_refinements_before_extend=0,operation=operation,version=0)
adaptiveCombiInstanceExtend.performSpatiallyAdaptiv(lmin=1,lmax=2,errorOperator=errorOperator,max_evaluations=22, do_plot=False)


adaptiveCombiInstanceExtend.print_resulting_combi_scheme("error_estimate_extend.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)
adaptiveCombiInstanceExtend.print_resulting_combi_scheme("error_estimate_extend.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False, consider_not_null=True)


# Dimension-wise

# rectilinear

In [ ]:
%matplotlib inline
import sparseSpACE
import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimension of the problem
dim = 2

# define integration domain boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function to be integrated
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**0 * (d+1) for d in range(dim)])
f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# reference integral solution for calculating errors
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=6, rebalancing=False)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,tol=0, max_evaluations=40, do_plot=False)

In [ ]:
adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("rectilinear_grids.pdf", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("rectilinear_grids.svg", show_border=True, fill_boundary_points=True, fontsize=60, ticks=False)


# 1D point sets

In [ ]:
%matplotlib inline
import sparseSpACE

import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**1 * (d+1) for d in range(dim)])
f = FunctionExpVar()
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

reference_solution = f.getAnalyticSolutionIntegral(a,b)
# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=6, rebalancing=False)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,tol=0.3*10**-1, max_evaluations=0, do_plot=False)

In [ ]:
adaptiveCombiInstanceSingleDim.draw_refinement_trees("refinement_tree_start.pdf", single_dim=0, fontsize=60)
adaptiveCombiInstanceSingleDim.draw_refinement_trees("refinement_tree_start.svg", single_dim=0, fontsize=60)
adaptiveCombiInstanceSingleDim.draw_refinement("refinement_start.pdf", single_dim=0, fontsize=60)
adaptiveCombiInstanceSingleDim.draw_refinement("refinement_start.svg", single_dim=0, fontsize=60)


In [ ]:
%matplotlib inline
import sparseSpACE

import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**1 * (d+1) for d in range(dim)])
f = FunctionExpVar()
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

reference_solution = f.getAnalyticSolutionIntegral(a,b)
# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=6, rebalancing=False)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,tol=0.3*10**-1, max_evaluations=22, do_plot=False)

In [ ]:
adaptiveCombiInstanceSingleDim.draw_refinement_trees("refinement_tree_refined.pdf", single_dim=0, fontsize=60)
adaptiveCombiInstanceSingleDim.draw_refinement_trees("refinement_tree_refined.svg", single_dim=0, fontsize=60)
adaptiveCombiInstanceSingleDim.draw_refinement("refinement_refined.pdf", single_dim=0, fontsize=60)
adaptiveCombiInstanceSingleDim.draw_refinement("refinement_refined.svg", single_dim=0, fontsize=60)


# Comparison of Strategies

In [ ]:
%matplotlib inline
import sparseSpACE

import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**1 * (d+1) for d in range(dim)])
f = FunctionExpVar()
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

reference_solution = f.getAnalyticSolutionIntegral(a,b)
# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=9, rebalancing=False, margin=0.9)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,10**-4, do_plot=False)

adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinementversion9_fine.pdf", show_border=True, markersize=3)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinementversion9_fine.svg", show_border=True, markersize=3)

In [ ]:
%matplotlib inline
import sparseSpACE

import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**1 * (d+1) for d in range(dim)])
f = FunctionExpVar()
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

reference_solution = f.getAnalyticSolutionIntegral(a,b)
# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=2, rebalancing=False, margin=0.9)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,10**-4, do_plot=False)

adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinementversion2_fine.pdf", show_border=True, markersize=3)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinementversion2_fine.svg", show_border=True, markersize=3)

In [ ]:
%matplotlib inline
import sparseSpACE

import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**1 * (d+1) for d in range(dim)])
f = FunctionExpVar()
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

reference_solution = f.getAnalyticSolutionIntegral(a,b)
# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=6, rebalancing=False, margin=0.9)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,10**-4, do_plot=False)

adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinementversion6_fine.pdf", show_border=True, markersize=3)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinementversion6_fine.svg", show_border=True, markersize=3)

# Example Strategy 3

In [ ]:
%matplotlib inline
import sparseSpACE
import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimension of the problem
dim = 2

# define integration domain boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function to be integrated
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**0 * (d+1) for d in range(dim)])
f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# reference integral solution for calculating errors
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=6, margin=0.9)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,4 * 10**-1, do_plot=False)

adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("standard_combi.pdf", show_border=True, markersize=20, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("standard_sparse_grid.pdf", show_border=True, markersize=40, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("standard_combi.svg", show_border=True, markersize=20, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("standard_sparse_grid.svg", show_border=True, markersize=40, ticks=False)
adaptiveCombiInstanceSingleDim.draw_refinement("standard_1D_refinements.pdf")
adaptiveCombiInstanceSingleDim.draw_refinement("standard_1D_refinements.svg")

adaptiveCombiInstanceSingleDim.continue_adaptive_refinement(3 * 10**-1)

adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("combi_refinement1.pdf", show_border=True, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinement1.pdf", show_border=True, markersize=40, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("combi_refinement1.svg", show_border=True, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinement1.svg", show_border=True, markersize=40, ticks=False)
adaptiveCombiInstanceSingleDim.draw_refinement("1D_refinements1.pdf")
adaptiveCombiInstanceSingleDim.draw_refinement("1D_refinements1.svg")

adaptiveCombiInstanceSingleDim.continue_adaptive_refinement(2 * 10**-1)

adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("combi_refinement2.pdf", show_border=True, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinement2.pdf", show_border=True, markersize=40, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("combi_refinement2.svg", show_border=True, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinement2.svg", show_border=True, markersize=40, ticks=False)
adaptiveCombiInstanceSingleDim.draw_refinement("1D_refinements2.pdf")
adaptiveCombiInstanceSingleDim.draw_refinement("1D_refinements2.svg")

adaptiveCombiInstanceSingleDim.continue_adaptive_refinement(1.9 * 10**-1)

adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("combi_refinement3.pdf", show_border=True, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinement3.pdf", show_border=True, markersize=40, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("combi_refinement3.svg", show_border=True, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_refinement3.svg", show_border=True, markersize=40, ticks=False)
adaptiveCombiInstanceSingleDim.draw_refinement("1D_refinements3.pdf")
adaptiveCombiInstanceSingleDim.draw_refinement("1D_refinements3.svg")

# Example Strategy 1

In [ ]:
%matplotlib inline
import sparseSpACE
import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimension of the problem
dim = 2

# define integration domain boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function to be integrated
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**0 * (d+1) for d in range(dim)])
f = GenzDiscontinious(border=midpoint,coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# reference integral solution for calculating errors
reference_solution = f.getAnalyticSolutionIntegral(a,b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=9, rebalancing=False)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,tol=0, max_evaluations=40, do_plot=False)

adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("comb_strategy_1.pdf", show_border=True, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_strategy_1.pdf", show_border=True, markersize=20, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme("comb_strategy_1.svg", show_border=True, ticks=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_strategy_1.svg", show_border=True, markersize=20, ticks=False)
adaptiveCombiInstanceSingleDim.draw_refinement("1D_refinements_strategy_1.pdf")
adaptiveCombiInstanceSingleDim.draw_refinement("1D_refinements_strategy_1.svg")

# rebalancing

In [ ]:
%matplotlib inline
import sparseSpACE

import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**1 * (d+1) for d in range(dim)])
f = FunctionExpVar()
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

reference_solution = f.getAnalyticSolutionIntegral(a,b)
# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=6, rebalancing=False)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,0.3*10**-1, do_plot=False)

adaptiveCombiInstanceSingleDim.draw_refinement_trees("refinement_tree_no_rebalance.pdf", single_dim=0, fontsize=60)
adaptiveCombiInstanceSingleDim.draw_refinement_trees("refinement_tree_no_rebalance.svg", single_dim=0, fontsize=60)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_no_rebalance.pdf", show_border=True, markersize=20)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_no_rebalance.svg", show_border=True, markersize=20)

In [ ]:
%matplotlib inline
import sparseSpACE

import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**1 * (d+1) for d in range(dim)])
f = FunctionExpVar()
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

reference_solution = f.getAnalyticSolutionIntegral(a,b)
# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=6, rebalancing=True)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,0.3*10**-1, do_plot=False)

adaptiveCombiInstanceSingleDim.draw_refinement_trees("refinement_tree_rebalance.pdf", single_dim=0, fontsize=60)
adaptiveCombiInstanceSingleDim.draw_refinement_trees("refinement_tree_rebalance.svg", single_dim=0, fontsize=60)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_rebalance.pdf", show_border=True, markersize=20)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid_rebalance.svg", show_border=True, markersize=20)

# Visual inspection

In [ ]:
%matplotlib inline
import sparseSpACE

import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**1 * (d+1) for d in range(dim)])
f = GenzC0(midpoint=midpoint, coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

reference_solution = f.getAnalyticSolutionIntegral(a,b)
# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=6, margin=0.5)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,10**-2, do_plot=False)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid2D_cont.svg", markersize=5, show_border=True)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid2D_cont.pdf", markersize=5, show_border=True)

In [ ]:
%matplotlib inline
import sparseSpACE

import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimensionality of problem
dim = 2

# integration boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function
midpoint = np.ones(dim) * 0.2
coefficients = np.array([ 10**0 * (d+1) for d in range(dim)])
f = GenzDiscontinious(border=midpoint, coeffs=coefficients)
# plot function
#f.plot(np.ones(dim)*a,np.ones(dim)*b)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)

reference_solution = f.getAnalyticSolutionIntegral(a,b)
# NEW! define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, operation=operation, version=6, margin=0.5)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1,2,errorOperator,10**-2, do_plot=False)

adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid2D_discont.svg", markersize=5, show_border=True)
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid("sparse_grid2D_discont.pdf", markersize=5, show_border=True)